In [1]:
import autograd.numpy as np
from autograd.core import primitive
from autograd import grad, jacobian, hessian
from autograd.numpy.numpy_grads import unbroadcast
import scipy.stats

In [2]:
foo = [1, 2, 3, 4, 5]
print foo[0:2]
print foo[2:5]

x = np.matrix([1., 3., 4.]).T
x_info = np.outer(0.1 * x, x) + np.eye(3)
mu = np.matrix([0.1, 0.2, 0.5]).T
mu_cov = np.matrix(np.outer(mu, mu) + np.eye(3))
print mu
print mu_cov
print np.concatenate((mu.A1, mu_cov.A1))

[1, 2]
[3, 4, 5]
[[ 0.1]
 [ 0.2]
 [ 0.5]]
[[ 1.01  0.02  0.05]
 [ 0.02  1.04  0.1 ]
 [ 0.05  0.1   1.25]]
[ 0.1   0.2   0.5   1.01  0.02  0.05  0.02  1.04  0.1   0.05  0.1   1.25]


In [3]:
x = np.array([1., 3., 5.])

def Mag(x):
    return np.dot(x.T, x)

MagGrad = grad(Mag)
MagGrad(x)

array([  2.,   6.,  10.])

In [10]:
def LogLikelihood(x_row, x_info, e_mu, e_mu_cov):
    # TODO: if you're not using autodiff you can just do the matrix multiply once.
    retval =  0.5 * (np.dot(e_mu.T, np.matmul(x_info, x_row)) + \
                     np.dot(x_row.T, np.matmul(x_info, e_mu)) - \
                     np.dot(e_mu.T, np.matmul(x_info, e_mu)) - \
                     np.trace(np.matmul(x_info, e_mu_cov)))
    return retval

@primitive
def LogLikelihoodAD(x_row, x_info, e_mu, e_mu_cov):
    return LogLikelihood(x_row, x_info, e_mu, e_mu_cov)

def LogLikelihoodGrad_e_mu(x_row, x_info, e_mu):
    return np.matmul(x_info, x_row - e_mu)

def LogLikelihoodGrad_e_mu_cov(x_info):
    return -0.5 * x_info

def LogLikelihoodAD_e_mu_vjp(g, ans, vs, gvs, x_row, x_info, e_mu, e_mu_cov):
    return g * LogLikelihoodGrad_e_mu(x_row, x_info, e_mu)

def LogLikelihoodAD_e_mu_cov_vjp(g, ans, vs, gvs, x_row, x_info, e_mu, e_mu_cov):
    return g * LogLikelihoodGrad_e_mu_cov(x_info)


LogLikelihoodAD.defvjp(LogLikelihoodAD_e_mu_vjp, argnum=2)
LogLikelihoodAD.defvjp(LogLikelihoodAD_e_mu_cov_vjp, argnum=3)

print LogLikelihood(x, x_info, mu, mu_cov)
print LogLikelihoodAD(x, x_info, mu, mu_cov)

x = np.array([1., 3., 4.])
x_info = np.outer(0.1 * x, x) + np.eye(3)
mu = np.array([0.1, 0.2, 0.5])
mu_cov = np.array(np.outer(mu, mu) + np.eye(3))

LogLikelihoodADGrad2 = grad(LogLikelihoodAD, argnum=2)
LogLikelihoodADGrad3 = grad(LogLikelihoodAD, argnum=3)
LogLikelihoodGrad2 = grad(LogLikelihood, argnum=2)
LogLikelihoodGrad3 = grad(LogLikelihood, argnum=3)

print LogLikelihoodADGrad2(x, x_info, mu, mu_cov)
print LogLikelihoodGrad2(x, x_info, mu, mu_cov)
print LogLikelihoodADGrad3(x, x_info, mu, mu_cov)
print LogLikelihoodGrad3(x, x_info, mu, mu_cov)

LogLikelihoodHess2 = hessian(LogLikelihood, argnum=2)
print LogLikelihoodHess2(x, x_info, mu, mu_cov)

LogLikelihoodHess3 = hessian(LogLikelihood, argnum=3)
print LogLikelihoodHess3(x, x_info, mu, mu_cov)


5.891
5.891
[  3.23   9.79  12.82]
[  3.23   9.79  12.82]
[[-0.55 -0.15 -0.2 ]
 [-0.15 -0.95 -0.6 ]
 [-0.2  -0.6  -1.3 ]]
[[-0.55 -0.15 -0.2 ]
 [-0.15 -0.95 -0.6 ]
 [-0.2  -0.6  -1.3 ]]
[[-1.1 -0.3 -0.4]
 [-0.3 -1.9 -1.2]
 [-0.4 -1.2 -2.6]]
[[[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]]


/home/rgiordan/Documents/git_repos/autograd/autograd/core.py:16: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


In [5]:

x = np.array([1., 3., 4.])
x_info = np.outer(0.1 * x, x) + np.eye(3)
mu = np.array([0.1, 0.2, 0.5]).T
mu_cov = np.array(np.outer(mu, mu) + np.eye(3))

print np.dot(x.T, np.matmul(x_info, mu))
print np.trace(np.matmul(x_info, mu_cov))


9.72
6.629


In [6]:
# It can handle matrices, but not matrix return types.
def MyFun(x):
    x_mat = np.matrix(x).T
    return 3 * np.dot(x_mat.T, x_mat)[0,0]

x = np.array([3., 2., 1.])
x_mat = np.matrix(x).T
print 3 * np.ravel(np.dot(x_mat.T, x_mat))
MyFunGrad = grad(MyFun)
MyFunGrad(x)

[ 42.]


array([ 18.,  12.,   6.])

In [43]:
def Wrapper(vec, beta):
    return(MyTwoArgFun(vec[0:2], vec[2:4], beta))

def WrapperRaw(vec, beta):
    return(MyTwoArgFunRaw(vec[0:2], vec[2:4], beta))

def MyTwoArgFunRaw(x, y, beta):
    return np.exp(0.1 * np.dot(beta.T, x) + 0.2 * np.dot(beta.T, y))

@primitive
def MyTwoArgFun(x, y, beta):
    return MyTwoArgFunRaw(x, y, beta)

@primitive
def MyTwoArgFunGrad_x(x, y, beta):
    return MyTwoArgFun(x, y, beta) * 0.1 * beta

@primitive
def MyTwoArgFunGrad_y(x, y, beta):
    return MyTwoArgFun(x, y, beta) * 0.2 * beta

def MyTwoArgFun_x_vjp(g, ans, vs, gvs, x, y, beta):
    return g * MyTwoArgFunGrad_x(x, y, beta)

def MyTwoArgFun_y_vjp(g, ans, vs, gvs, x, y, beta):
    return g * MyTwoArgFunGrad_y(x, y, beta)

MyTwoArgFun.defvjp(MyTwoArgFun_x_vjp, argnum=0)
MyTwoArgFun.defvjp(MyTwoArgFun_y_vjp, argnum=1)

def MyTwoArgFunHess_x_x(x, y, beta):
    return MyTwoArgFun(x, y, beta) * 0.1 * 0.1 * np.outer(beta, beta)

def MyTwoArgFunHess_x_y(x, y, beta):
    return MyTwoArgFun(x, y, beta) * 0.1 * 0.2 * np.outer(beta, beta)

def MyTwoArgFunHess_y_x(x, y, beta):
    return MyTwoArgFunHess_x_y(x, y, beta).T

def MyTwoArgFunHess_y_y(x, y, beta):
    return MyTwoArgFun(x, y, beta) * 0.2 * 0.2 * np.outer(beta, beta)

def MyTwoArgFunGrad_x_x_vjp(g, ans, vs, gvs, x, y, beta):
    print g.shape
    return np.matmul(MyTwoArgFunHess_x_x(x, y, beta), g)

def MyTwoArgFunGrad_x_y_vjp(g, ans, vs, gvs, x, y, beta):
    return np.matmul(MyTwoArgFunHess_x_y(x, y, beta), g)

def MyTwoArgFunGrad_y_y_vjp(g, ans, vs, gvs, x, y, beta):
    return np.matmul(MyTwoArgFunHess_y_y(x, y, beta), g)

def MyTwoArgFunGrad_y_x_vjp(g, ans, vs, gvs, x, y, beta):
    return np.matmul(MyTwoArgFunHess_y_x(x, y, beta), g)

MyTwoArgFunGrad_x.defvjp(MyTwoArgFunGrad_x_x_vjp, argnum=0)
MyTwoArgFunGrad_x.defvjp(MyTwoArgFunGrad_x_y_vjp, argnum=1)
MyTwoArgFunGrad_y.defvjp(MyTwoArgFunGrad_y_x_vjp, argnum=0)
MyTwoArgFunGrad_y.defvjp(MyTwoArgFunGrad_y_y_vjp, argnum=1)


x = np.array([3., 2. ])
y = x + 5.
beta = np.array([1.1, 2.4]) * 0.1
print np.concatenate((x, y))

WrapperHess = hessian(Wrapper)
WrapperRawHess = hessian(WrapperRaw)
print WrapperHess(np.concatenate((x, y)), beta) - WrapperRawHess(np.concatenate((x, y)), beta)

[ 3.  2.  8.  7.]
(2,)
(2,)
(2,)
(2,)
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -5.42101086e-20   0.00000000e+00  -1.08420217e-19   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -1.08420217e-19   0.00000000e+00  -2.16840434e-19   0.00000000e+00]]


In [38]:
print beta
print beta.T
print np.outer(beta, beta.T)
print np.outer(beta, beta)
print np.dot(beta, beta) # WTF
print np.dot(beta, beta.T) # WTF
print sum(beta * beta)

[ 0.11  0.24]
[ 0.11  0.24]
[[ 0.0121  0.0264]
 [ 0.0264  0.0576]]
[[ 0.0121  0.0264]
 [ 0.0264  0.0576]]
0.0697
0.0697
0.0697
